In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
from scipy import stats
from math import pi
from scipy.spatial.distance import cdist
import time

from orbit.models.ktrx import KTRXFull, KTRXAggregated
from orbit.models.ktrlite import KTRLiteMAP
from orbit.estimators.pyro_estimator import PyroEstimatorVI, PyroEstimatorMAP
from orbit.estimators.stan_estimator import StanEstimatorMAP
from orbit.diagnostics.metrics import smape
from orbit.utils.features import make_fourier_series_df, make_fourier_series
from orbit.diagnostics.plot import plot_predicted_data


plt.style.use('fivethirtyeight')

%load_ext autoreload
%autoreload 2

In [5]:
def sim_data_seasonal(n, RS):
    np.random.seed(RS)
    # make the time varing coefs  
    tau = np.arange(1, n+1)/n
    data = pd.DataFrame({
        'tau': tau,
        'date': pd.date_range(start='1/1/2018', periods=n),
        'beta1': 2 * tau,
        'beta2': np.sin(2*pi*tau),
        'beta3': np.sin(4*pi*(tau-1/8)),
        'x1': stats.chi2.rvs(4, size=n),
        'x2': stats.t.rvs(2, size=n),
        'x3': stats.t.rvs(2, size=n),
        'trend': np.cumsum(np.concatenate((np.array([1]), np.random.normal(0, 0.1, n-1)))),
        'error': np.random.normal(0, 1, size=n) #stats.t.rvs(30, size=n),#
    })
    
    # add error to the data 
    #err_cov = np.exp(-cdist(data.tau.values.reshape(n, -1), data.tau.values.reshape(n, -1), 'euclid')/10)
    #L = np.linalg.cholesky(err_cov).T
    #data['error2'] = L.dot(stats.chi2.rvs(100, size=n))
    
    data['y'] = data.x1 * data.beta1 + data.x2 * data.beta2 + data.x3 * data.beta3 + data.error
    #data['y2'] = data.x1 * data.beta1 + data.x2 * data.beta2 + data.x3 * data.beta3 + data.error2
    #data['y3'] = data.trend + data.x1 * data.beta1 + data.x2 * data.beta2 + data.x3 * data.beta3 + data.error
    return(data)

def sim_data_rw(n, RS, p=3):
    np.random.seed(RS)

    # initializing coefficients at zeros, simulate all coefficient values
    lev = np.cumsum(np.concatenate((np.array([5.0]), np.random.normal(0, 0.01, n-1))))
    beta = np.concatenate(
        [np.random.uniform(0.05, 0.12, size=(1,p)),
         np.random.normal(0, 0.01, size=(n-1,p))], 
            axis=0)
    beta = np.cumsum(beta, 0)

    # simulate regressors
    covariates = np.random.normal(0, 1, (n, p))

    # observation with noise
    y = lev + (covariates * beta).sum(-1) + 0.3 * np.random.normal(0, 1, n)

    regressor_col = ['x{}'.format(pp) for pp in range(1, p+1)]
    data = pd.DataFrame(covariates, columns=regressor_col)
    data['y'] = y
    data['date'] = pd.date_range(start='1/1/2018', periods=len(y))
    
    # hack for p = 3 
    data['beta1'] = (covariates * beta)[:,0]
    data['beta2'] = (covariates * beta)[:,1]
    data['beta3'] = (covariates * beta)[:,2]    
    
    return(data)

def multiple_test(N,n, sim_type):
    out = pd.DataFrame()
    out['index'] = range(0, N)
    # for hte model fit 
    out['time_1'] = 0.0
    out['time_2'] = 0.0
    
    out['SSE_1'] = 0.0
    out['SSE_2'] = 0.0
    out['RMSE_1'] = 0.0
    out['RMSE_2'] = 0.0
    out['max_error_1'] = 0.0
    out['max_error_2'] = 0.0
    
    # for the true values 
    out['SSE_beta1_1'] = 0.0
    out['SSE_beta1_2'] = 0.0 
    out['SSE_beta2_1'] = 0.0   
    out['SSE_beta2_2'] = 0.0
    out['SSE_beta3_2'] = 0.0
    out['SSE_beta3_1'] = 0.0
    
    for i in range(0, N):
        # simulate the data 
        if sim_type == 'sea':
            data = sim_data_seasonal(n = n, RS = 1000+i)
       
        if sim_type == 'rw':
            data = sim_data_rw(n = n, RS = 1000+i, p=3)    
    
        #print(data.head())
    
        # define stuff 
        regressor_col=['x1', 'x2', 'x3']
        response_col = 'y'
        # run the model 
        # run lite first 
        ktr_lite = KTRLiteMAP(
        response_col=response_col,
        date_col='date',
        level_knot_scale=1,
        seed=2000+i,
        span_level= .1, 
        estimator_type=StanEstimatorMAP,
        )
        ktr_lite.fit(df=data)
        level_knots_stan = ktr_lite._aggregate_posteriors['map']['lev_knot'][0]
        level_knot_dates = ktr_lite._level_knot_dates
        level_knots_stan = np.array([0] * len(level_knot_dates))
        
        # run full model second 
        # there are two of these 
        ktrx1 = KTRXFull(
        response_col=response_col,
        date_col='date',

        degree_of_freedom=30,
        level_knot_scale=.001,
        level_knot_dates=level_knot_dates,
        level_knots=level_knots_stan,

        regressor_col=regressor_col,
        #regressor_sign=['='] * len(regressor_col),
        regressor_knot_pooling_loc=[0] * len(regressor_col),
        regressor_knot_pooling_scale=[1] * len(regressor_col),
        regressor_knot_scale=[.2] * len(regressor_col),
        # seasonal_knots_input={},

        span_coefficients=0.1,
        rho_coefficients=0.1, #.1,.11,.12
        prediction_percentiles=[2.5, 97.5],
        seed=2000+i,
        num_steps=1000,
        num_particles=100,
        num_sample=1000,
        learning_rate=0.01,
        learning_rate_total_decay=1.,
        verbose=False,
        message=100,
        n_bootstrap_draws=-1,
        estimator_type=PyroEstimatorVI,
        mvn=1
        )

        ############################
        # second one 
        ktrx2 = KTRXFull(
        response_col=response_col,
        date_col='date',

        degree_of_freedom=30,
        level_knot_scale=1,
        level_knot_dates=level_knot_dates,
        level_knots=level_knots_stan,

        regressor_col=regressor_col,
        regressor_sign=['='] * len(regressor_col),
        regressor_knot_pooling_loc=[0] * len(regressor_col),
        regressor_knot_pooling_scale=[1] * len(regressor_col),
        regressor_knot_scale=[.2] * len(regressor_col),
        # seasonal_knots_input={},

        span_coefficients=0.1,
        rho_coefficients=0.1, #.1,.11,.12
        prediction_percentiles=[2.5, 97.5],
        seed=3000+i, # just to make them different from each other
        num_steps=1000,
        num_particles=100,
        num_sample=1000,
        learning_rate=0.01,
        learning_rate_total_decay=1.,
        verbose=False,
        message=100,
        n_bootstrap_draws=-1,
        estimator_type=PyroEstimatorVI,
        )
        
        # fit the models and recod the times
        start_time = time.time()
        ktrx1.fit(df=data)
        time_1 = time.time() - start_time
        
        start_time = time.time()
        ktrx2.fit(df=data)
        time_2 = time.time() - start_time
        
        # get the predictions 
        predicted_df_1 = ktrx1.predict(df=data)
        predicted_df_2 = ktrx2.predict(df=data)
        

        
        # compare to observations  
        SSE_1 = sum((predicted_df_1['prediction'] - data['y'])**2.0 )
        SSE_2 = sum((predicted_df_2['prediction'] - data['y'])**2.0 )
        
        max_misfit_1 = max(abs(predicted_df_1['prediction'] - data['y']) )
        max_misfit_2 = max(abs(predicted_df_2['prediction'] - data['y']) )
    
        out.at[i, 'time_1'] = time_1 
        out.at[i, 'time_2'] = time_2 
    
        out.at[i, 'SSE_1'] = SSE_1 
        out.at[i, 'SSE_2'] = SSE_2 
        
        out.at[i, 'RMSE_1'] = (SSE_1/n)**(0.5) 
        out.at[i, 'RMSE_2'] = (SSE_2/n)**(0.5) 
        
        
        out.at[i, 'max_error_1'] = max_misfit_1
        out.at[i, 'max_error_2'] = max_misfit_2
        
        #compare to true values 
        coef_df_1= ktrx1.get_regression_coefs(
        aggregate_method='median',
        include_ci=False)
        
        coef_df_2= ktrx2.get_regression_coefs(
        aggregate_method='median',
        include_ci=False)
        
        SSE_beta1_1 = sum((coef_df_1['x1']-data['beta1'])**2.0)
        SSE_beta2_1 = sum((coef_df_1['x2']-data['beta2'])**2.0)
        SSE_beta3_1 = sum((coef_df_1['x3']-data['beta3'])**2.0)
        SSE_beta1_2 = sum((coef_df_2['x1']-data['beta1'])**2.0)
        SSE_beta2_2 = sum((coef_df_2['x2']-data['beta2'])**2.0)
        SSE_beta3_2 = sum((coef_df_2['x3']-data['beta3'])**2.0)    
        
        out.at[i,'SSE_beta1_1'] = SSE_beta1_1
        out.at[i,'SSE_beta2_1'] = SSE_beta2_1
        out.at[i,'SSE_beta3_1'] = SSE_beta3_1
        out.at[i,'SSE_beta1_2'] = SSE_beta1_2
        out.at[i,'SSE_beta2_2'] = SSE_beta2_2
        out.at[i,'SSE_beta3_2'] = SSE_beta3_2
        
    return(out)
        
        
    

In [ ]:
multiple_test(N=10,n=300, sim_type='sea')

INFO:root:Guessed max_plate_nesting = 1


In [ ]:
multiple_test(N=3,n=300, sim_type='rw')

In [ ]:
sim_data_rw(n=10, RS=8888, p=3)

In [ ]:
k

In [ ]:
1